In [1]:
import pandas as pd
import re
from random import random
from numpy import array
from numpy import cumsum

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import SnowballStemmer
from nltk.tokenize import TweetTokenizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Bidirectional
from keras.layers import LSTM,SpatialDropout1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data=pd.read_csv(r"F:\FYP-II-master\data\training.csv")
data.head()

,class,text,Unnamed: 2
0,PI,What's the difference between the iPhone X a...,NaN
1,no,@ siracusa just listened to Hypercritical #16...,NaN
2,PI,Same. IPhone X won't be bad https:// twitt...,NaN
3,no,Ive had the iphone 8 plus and iphone x and i...,NaN
4,no,“ iphone x for £150” do i have dickhead writ...,NaN


In [3]:
data=data.drop(columns=['Unnamed: 2'])

In [4]:
data['class']=data['class'].replace(['Undefined','undefined'],'un')
data=data[data['class']!='un']
data.head(10)

,class,text
0,PI,What's the difference between the iPhone X a...
1,no,@ siracusa just listened to Hypercritical #16...
2,PI,Same. IPhone X won't be bad https:// twitt...
3,no,Ive had the iphone 8 plus and iphone x and i...
4,no,“ iphone x for £150” do i have dickhead writ...
5,PI,This iPhone X still very sexy a year later
7,PI,If not I'm using my kidney. Omo what is ferrar...
8,PI,iPhone X 's max for sale
9,no,Someone once told me that.....\n\nBut I've nev...
10,PI,screenshots from the iphone x look so clapped


In [5]:
replace_space=re.compile('[/(){}\[\]\|@,;]')
replace_bad_symbol=re.compile('[^0-9a-z #+_]')
Stopwords=set(stopwords.words('english'))

In [6]:
def clean_text(text):
    text=text.lower()
    text=replace_space.sub(' ',text)
    text=replace_bad_symbol.sub('',text)

    text=' '.join(word for word in text.split() if word not in Stopwords)
    return text

data['text']=data['text'].apply(clean_text)
data.head(10)

,class,text
0,PI,whats difference iphone x iphone x x max
1,no,siracusa listened hypercritical #16 whole epis...
2,PI,iphone x wont bad https twittercom afuaasam st...
3,no,ive iphone 8 plus iphone x id say iphone 8 plu...
4,no,iphone x 150 dickhead written
5,PI,iphone x still sexy year later
7,PI,im using kidney omo ferrari buggati iphone x m...
8,PI,iphone x max sale
9,no,someone told thatbut ive never seen ios turned...
10,PI,screenshots iphone x look clapped


In [7]:
t=Tokenizer(num_words=50000,lower=True)
t.fit_on_texts(data['text'])
x=t.texts_to_sequences(data['text'])
x=sequence.pad_sequences(x,maxlen=250)
x.shape

(906, 250)

In [8]:
data['class']=data['class'].replace(['no','No'],0)
data['class']=data['class'].replace(['Pl','PI','yes','Yes'],1)
data.head(10)

,class,text
0,1,whats difference iphone x iphone x x max
1,0,siracusa listened hypercritical #16 whole epis...
2,1,iphone x wont bad https twittercom afuaasam st...
3,0,ive iphone 8 plus iphone x id say iphone 8 plu...
4,0,iphone x 150 dickhead written
5,1,iphone x still sexy year later
7,1,im using kidney omo ferrari buggati iphone x m...
8,1,iphone x max sale
9,0,someone told thatbut ive never seen ios turned...
10,1,screenshots iphone x look clapped


In [9]:
y=pd.get_dummies(data['class']).values
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1)
print(x_train.shape,x_test.shape)
print(y_train.shape,y_test.shape)
y

(815, 250) (91, 250)
(815, 2) (91, 2)


array([[0, 1],
       [1, 0],
       [0, 1],
       ...,
       [1, 0],
       [1, 0],
       [1, 0]], dtype=uint8)

In [10]:
model=Sequential()
model.add(Embedding(50000,100,input_length=x.shape[1]))
model.add(Bidirectional(LSTM(100,dropout=0.2)))
model.add(Dense(2,activation='sigmoid'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

fit=model.fit(x_train,y_train,epochs=5,batch_size=64,validation_split=0.1)

Epoch 1/5
12/12 [==============================] - 31s 802ms/step - loss: 0.6793 - accuracy: 0.5437 - val_loss: 0.6471 - val_accuracy: 0.6341
Epoch 2/5
12/12 [==============================] - 10s 836ms/step - loss: 0.6468 - accuracy: 0.6179 - val_loss: 0.6272 - val_accuracy: 0.6341
Epoch 3/5
12/12 [==============================] - 11s 955ms/step - loss: 0.5871 - accuracy: 0.6468 - val_loss: 0.5396 - val_accuracy: 0.8049
Epoch 4/5
12/12 [==============================] - 10s 872ms/step - loss: 0.4618 - accuracy: 0.8730 - val_loss: 0.6035 - val_accuracy: 0.7317
Epoch 5/5
12/12 [==============================] - 10s 849ms/step - loss: 0.3925 - accuracy: 0.9001 - val_loss: 0.6093 - val_accuracy: 0.7317


In [11]:
acr=model.evaluate(x_test,y_test)

3/3 [==============================] - 0s 65ms/step - loss: 0.5844 - accuracy: 0.7143


In [12]:
# # create a sequence classification instance
# def get_sequence(n_timesteps):
#     # create a sequence of random numbers in [0,1]
#     X = array([random() for _ in range(n_timesteps)])
#     # calculate cut-off value to change class values
#     limit = n_timesteps/4.0
#     # determine the class outcome for each item in cumulative sequence
#     y = array([0 if x < limit else 1 for x in cumsum(X)])
#     # reshape input and output data to be suitable for LSTMs
#     X = X.reshape(1, n_timesteps, 1)
#     y = y.reshape(1, n_timesteps, 1)
#     return X, y

# # define problem properties
# n_timesteps = 10
# # define LSTM
# model = Sequential()
# model.add(Bidirectional(LSTM(100, return_sequences=True), input_shape=(n_timesteps, 1)))
# model.add(TimeDistributed(Dense(1, activation='sigmoid')))
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# # train LSTM
# for epoch in range(1000):
#     # generate new random sequence
#     X,y = get_sequence(n_timesteps)
#     # fit model for one epoch on this sequence
#     model.fit(X, y, epochs=1, batch_size=1, verbose=2)
#     # evaluate LSTM
# X,y = get_sequence(n_timesteps)
# yhat = model.predict_classes(X, verbose=0)
# for i in range(n_timesteps):
#     print('Expected:', y[0, i], 'Predicted', yhat[0, i])